In [ ]:
#Function occupancyChecker checks if a smooth path runs through an occupied space and returns the location of occurance
#Assumptions
# An occupancy function exists that is called as follows occpied = occupancyFunc(x,y,z)
# Path is in absolute reference frame
# Solution has coefficient vectors of the same length
#Inputs/Needed Variables
# solution - an object containing points, and times related to path solution
# occupancyFunc - occpied = occupancyFunc(x,y,z) where occupied is either a bool and x, y, z are positions of the path
# grid_resx - the resolution of the grid in the x direction
# grid_resy - the resolution of the grid in the y direction
# grid_resz - the resolution of the grid in the z direction
# dim - the dimension of the path to be checked
# aggressParam - a parameter of how aggressive to check a path (0,infty), Closer to 0 means check every point on the path, Closer to infinity => check no points
# coeffMat - hold coefficients and allows for looping and selection
#Outputs/Things to Find
# did_pass - a boolean that is true when the path doesn't hit anything
# timeHit - the time when the path goes through occupied space
# pointsBetweem - a vector of the points between which the occupation collision occurs

#Read in the coefficients into a matrix
coeffMat = [solution.x_coeffs; solution.y_coeffs; solution.z_coeffs];
#Initialize/read in the resolution variables
grid_resx = 0.05;
grid_resy = 0.05;
grid_resz = 0.05;
#Calculate the distance time to travel in each direction before checking occupancy using 1st order taylor series approximation
#Loop for 2 or 3 dimensions if dim >3
#evaluate the polynomial first direvative at current time
#find the required delta_t's needed to go the set distance before checking occupancy
#determine the smallest one
#move the smallest one, and decrease the delta t's for the others
#evaluate occupancy at current point
#evaluate the new delta_t for the dimension
#Check if at the past the end time and while loop if not passed it
#determine the smallest one
#move the smallest one, and decrease the delta t's for the others
#evaluate occupancy at current point
#if occupancy is one break the loop
#evaluate the new delta_t for the dimension
#Out of loop check if current time is at or above the final time
#If not find the nearest points forward and backward
#return the points in hopes that a point will be added between the returned points
#Loop though the dimension passed
